In [5]:
import torch
import torch.nn as nn
from networks import AttnDecoderRNN, CNN

In [6]:
class CAM_normD(nn.Module):
    def __init__(self,nclass,channel_size,hidden_size,output_size,dropout_p=0.1,max_length = 64,D_ch =16, nWriter = 1300,iam = False):
        super(CAM_normD,self).__init__()
        self.encoder =CNN(channel_size)
        self.hidden_size = hidden_size
        
        self.decoder_forradical = AttnDecoderRNN(hidden_size, output_size, dropout_p,max_length)
        # self.decoderfeat_forradical =nn.Sequential(
        #     nn.Conv2d(256, 128, 3, 1, 1), nn.InstanceNorm2d(128), nn.PReLU(), nn.AvgPool2d(2, 2),
        #     nn.Conv2d(128, 64, 3, 1, 1), nn.InstanceNorm2d(128), nn.PReLU(), nn.MaxPool2d(2, 2),
        #     nn.Conv2d(64, 16, 3, 1, 1), nn.InstanceNorm2d(16), nn.PReLU(),
        #     nn.Conv2d(16, 1, 3, 1, 1)
        #     )        

        self.decoderfeat_forradical0 =nn.Sequential(
            nn.Conv2d(256, 128, 3, 1, 1), nn.InstanceNorm2d(128), nn.PReLU(), nn.AvgPool2d(2, 2),
            nn.Conv2d(128, 64, 3, 1, 1)
            )        

        self.decoderfeat_forradical1 =nn.Sequential(
            nn.InstanceNorm2d(128), nn.PReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 16, 3, 1, 1), nn.InstanceNorm2d(16), nn.PReLU(),
            nn.Conv2d(16, 1, 3, 1, 1)
            )        

        

    def initHidden(batch_size,hidden_size):
        result = torch.autograd.Variable(torch.zeros(1, batch_size, hidden_size))
        return result

    def forward(self, image, text_radical, length_radical):
        
        encode = self.encoder(image)
        b, c, _, _ = encode.size() #batch,256
        
        # out, bottleneck_out = self.unetD(image)
        out = self.D(image)
        loss_forradical,new_encode= self.decoder_forradical(encode,image,text_radical,length_radical)
        # pred_radical = self.decoderfeat_forradical(new_encode)
        pred_radical0 = self.decoderfeat_forradical0(new_encode)
        pred_radical1 = self.decoderfeat_forradical1(pred_radical0)
        pred_radical = (pred_radical0, pred_radical1)
        
        return pred_radical, loss_forradical, out

In [10]:
from options import TrainOptions
opt = TrainOptions()
from x_dataset import CGGANDataset

dataset = CGGANDataset(args=opt)
train_loader = torch.utils.data.DataLoader(
    dataset, batch_size=opt.batch_size,
    shuffle=True, sampler=None, drop_last=True, num_workers=int(opt.num_threads))

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/ai_font/data/pickle/letter2font.pickle'

In [8]:
nclass = opt.n_alphabet+1
channel_size = opt.input_nc
hidden_size = opt.hidden_size
output_size = opt.n_alphabet+2
dropout_p = opt.dropout_p
max_length = opt.max_length
D_ch = opt.D_ch
nWriter = opt.num_writer
norm = opt.norm
init_type = opt.init_type 
init_gain = opt.init_gain
iam = False

In [9]:
camD = CAM_normD(nclass,channel_size,hidden_size,output_size,dropout_p,max_length,D_ch,nWriter,iam)